In [ ]:
import json
import os
# read json file
with open('../Data/stations.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

for station_name, station_info in data.items():
    print("Station name:", station_name)
    print("district:", station_info['district'])
    print("introduce:", station_info['intro'])

    # print bus info
    print("bus info:")
    for bus_info in station_info['bus_info']:
        print("- chukou:", bus_info['chukou'])
        for bus_out_info in bus_info['busOutInfo']:
            print("  - busInfo:", bus_out_info['busInfo'])
            print("  - busName:", bus_out_info['busName'])

    # print outt info
    print("outt info:")
    for out_info in station_info['out_info']:
        print("- outt:", out_info['outt'])
        print("- textt:", out_info['textt'])

    print("\n")


# read JSON file
with open('../Data/cards.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

for record in data:
    print("code:", record['code'])
    print("money:", record['money'])
    print("create_time:", record['create_time'])



import psycopg
import os
db_name = os.environ['DB_NAME']
db_user = os.environ['DB_USER']
db_password = os.environ['DB_PASSWORD']
db_host = os.environ['DB_HOST']
db_port = os.environ['DB_PORT']
print(db_name, db_user, db_password, db_host, db_port)
conn = psycopg.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
class DatabaseManipulation:
    def __init__(self):
        self.conn = conn
        self.cur = self.conn.cursor()
    def findMovieById(self, id):
        query = "SELECT m.title, \
                c.country_name, \
                m.year_released, \
                m.runtime FROM movies m JOIN countries c ON m.country = c.country_code \
                WHERE m.movieid = %s"
        self.cur.execute(query, (id,))
        result = self.cur.fetchone()
        return result
    
q = DatabaseManipulation()
q.cur = conn.cursor()
print(q.findMovieById(2))


In [14]:
import json
import psycopg
import os
import subprocess
import re

def extract_env_vars():
    with open('../setenv.sh', 'r') as file:
        lines = file.readlines()
    env_vars = {}
    for line in lines:
        match = re.match(r"export (\w+)='?([^']*)'?", line)
        if match:
            env_vars[match.group(1)] = match.group(2).strip()
    print(env_vars)
    return env_vars

# Function to connect to the PostgreSQL database
def create_conn():
    env_vars = extract_env_vars()
    db_name = env_vars['DB_NAME']
    db_user = env_vars['DB_USER']
    db_password = env_vars['DB_PASSWORD']
    db_host = env_vars['DB_HOST']
    db_port = env_vars['DB_PORT']
    conn = psycopg.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
    return conn

# Function to create tables in the PostgreSQL database
def create_tables(conn):
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS stations (
            station_name TEXT PRIMARY KEY,
            district TEXT,
            intro TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS bus_info (
            station_name TEXT,
            chukou TEXT,
            bus_name TEXT,
            bus_info TEXT,
            FOREIGN KEY (station_name) REFERENCES stations (station_name)
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS out_info (
            station_name TEXT,
            outt TEXT,
            textt TEXT,
            FOREIGN KEY (station_name) REFERENCES stations (station_name)
        )
    """)
    conn.commit()

# Function to insert data into the tables
def insert_data(conn, data):
    cur = conn.cursor()
    for station_name, station_info in data.items():
        cur.execute("""
            INSERT INTO stations (station_name, district, intro) 
            VALUES (%s, %s, %s)
        """, (station_name, station_info['district'], station_info['intro']))
        for bus_info in station_info['bus_info']:
            for bus_out_info in bus_info['busOutInfo']:
                cur.execute("""
                    INSERT INTO bus_info (station_name, chukou, bus_name, bus_info) 
                    VALUES (%s, %s, %s, %s)
                """, (station_name, bus_info['chukou'], bus_out_info['busName'], bus_out_info['busInfo']))
        for out_info in station_info['out_info']:
            cur.execute("""
                INSERT INTO out_info (station_name, outt, textt) 
                VALUES (%s, %s, %s)
            """, (station_name, out_info['outt'], out_info['textt']))
    conn.commit()

# Read the JSON file
with open('../Data/stations.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Call the function to create tables
conn = create_conn()
create_tables(conn)

# Call the function to insert data
insert_data(conn, data)

{'DB_NAME': 'cscs307', 'DB_USER': 'superlucky', 'DB_PASSWORD': '', 'DB_HOST': 'localhost', 'DB_PORT': '5432'}


In [8]:
import os
import psycopg
import json
import time

def print_progress(percentage):
    pbstr = "||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||"
    pbwidth = 60
    val = percentage * 100
    lpad = int(percentage * pbwidth)
    rpad = pbwidth - lpad
    print("\r{:.1f}% [{}{}]".format(val, pbstr[:lpad], ' ' * rpad), end='', flush=True)

def main():
    data_path = "../Data"

    # Database connection string
    db_name = os.getenv("DB_NAME")
    db_user = os.getenv("DB_USER")
    db_password = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT")
    conn_str = f"dbname={db_name} user={db_user} password={db_password} host={db_host} port={db_port}"

    print("Parsing JSON files...")

    start = time.time()

    # Read from rides.json
    with open(f"{data_path}/rides.json", 'r') as file:
        rides = json.load(file)

    end = time.time()

    print(f"JSON parsing completed in {end - start} seconds.")

    # Connect to the database
    conn = psycopg.connect(conn_str)
    cur = conn.cursor()

    print("Inserting data into the database...")
    start = time.time()

    # Loop over each instance in the rides array
    total = len(rides)
    print(f"Total rides: {total}")
    for i, ride in enumerate(rides):
        # Extract data from JSON
        user = ride["user"]
        start_station = ride["start_station"]
        end_station = ride["end_station"]
        price = ride["price"]
        start_time = ride["start_time"]
        end_time = ride["end_time"]
        try:
            # Execute the prepared statement
            cur.execute("INSERT INTO rides (rail_user, start_station, end_station, price, start_time, end_time) VALUES (%s, %s, %s, %s, %s, %s)",
                        (user, start_station, end_station, price, start_time, end_time))
            print_progress((i + 1) / total)
        except Exception as e:
            print(e)

    conn.commit()

    end = time.time()
    print(f"\nData insertion completed in {end - start} seconds.")

if __name__ == "__main__":
    main()

Parsing JSON files...
JSON parsing completed in 0.15045404434204102 seconds.


OperationalError: [Errno 8] nodename nor servname provided, or not known

In [ ]:
import json
import psycopg
import os

# Function to print progress
def print_progress(percentage):
    val = percentage * 100
    lpad = int(percentage * 60)
    rpad = 60 - lpad
    print(f"\r{val:.1f}% [{'=' * lpad}{' ' * rpad}]", end='')

db_name = os.environ['DB_NAME']
db_user = os.environ['DB_USER']
db_password = os.environ['DB_PASSWORD']
db_host = os.environ['DB_HOST']
db_port = os.environ['DB_PORT']
print(db_name, db_user, db_password, db_host, db_port,'\n')

# Create a connection to the database
conn = psycopg.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)

cur = conn.cursor()

# Function to execute SQL commands from a string
def execute_sql(sql_commands):
    cur.execute(sql_commands)

# Function to load JSON from a file
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Read from JSON files (Assuming all these files are in the same directory as the script)
cards = load_json('cards.json')
passengers = load_json('passengers.json')
rides = load_json('rides.json')
lines = load_json('lines.json')
stations = load_json('stations.json')

# Create the tables
execute_sql("""
CREATE TABLE IF NOT EXISTS cards (
    code VARCHAR(9) PRIMARY KEY,
    money MONEY NOT NULL,
    create_time TIMESTAMP NOT NULL);
""")
execute_sql("""
CREATE TABLE IF NOT EXISTS passengers (
    name VARCHAR(255) NOT NULL,
    id_number VARCHAR(18) PRIMARY KEY NOT NULL,
    phone_number VARCHAR(11) NOT NULL,
    gender VARCHAR(255) NOT NULL,
    district VARCHAR(255) NOT NULL);
""")
execute_sql("""
CREATE TABLE IF NOT EXISTS rides (
    rail_user VARCHAR(255),
    start_station VARCHAR(255),
    end_station VARCHAR(255),
    price DOUBLE PRECISION,
    start_time TIMESTAMP,
    end_time TIMESTAMP);
""")
# Continue creating all necessary tables...

# Insert data into the tables
def insert_data(table, data):
    for record in data:
        keys = ', '.join(record.keys())
        values = ', '.join([f"'{str(val).replace("'", "''")}'" for val in record.values()])
        cur.execute(f"INSERT INTO {table} ({keys}) VALUES ({values}) ON CONFLICT DO NOTHING;")

# Call the insert function for each table
insert_data('cards', cards)
insert_data('passengers', passengers)
insert_data('rides', rides)
# Continue inserting data for all tables...

# Commit the changes
conn.commit()

# Close the connection
cur.close()
conn.close()


In [1]:

using json = nlohmann::json;


void printProgress(double percentage) {
    double val = percentage * 100;
    int lpad = (int) (percentage * PBWIDTH);
    int rpad = PBWIDTH - lpad;
    printf("\r%4.1f%% [%.*s%*s]", val, lpad, PBSTR, rpad, "");
    fflush(stdout);
}


int main() {
    const char* data_path_cstr = getenv("DATA_PATH");
    std::string data_path = data_path_cstr ? data_path_cstr : "";

    // Database connection string
    const char* db_name_cstr = getenv("DB_NAME");
    std::string db_name = db_name_cstr ? db_name_cstr : "postgres";

    const char* db_user_cstr = getenv("DB_USER");
    std::string db_user = db_user_cstr ? db_user_cstr : "postgres";

    const char* db_password_cstr = getenv("DB_PASSWORD");
    std::string db_password = db_password_cstr ? db_password_cstr : "";

    const char* db_host_cstr = getenv("DB_HOST");
    std::string db_host = db_host_cstr ? db_host_cstr : "localhost";

    const char* db_port_cstr = getenv("DB_PORT");
    std::string db_port = db_port_cstr ? db_port_cstr : "5432";

    std::string conn_str = "dbname=" + db_name + " user=" + db_user + " password=" + db_password + " host=" + db_host + " port=" + db_port;

    // CSV path, /tmp by default
    const char* csv_path_ctsr = getenv("CSV_PATH");
    std::string csv_path = csv_path_ctsr ? csv_path_ctsr : "?";
    const std::filesystem::path current_path = std::filesystem::path(csv_path);

    std::wstring_convert<std::codecvt_utf8<wchar_t>, wchar_t> converter;

    // Chinese string
    const std::string chu1_ru4_kou3 = "出入口";

    // timing
    // std::time_t start, end;
    struct timeval start, end;
    double time_spent;

    std::cout << "Parsing JSON files..." << std::endl;

    // start = std::time(nullptr);
    gettimeofday(&start, NULL);

    // Read from cards.json
    std::ifstream file(data_path + "/cards.json");
    json cards;
    file >> cards;

    // Read from passengers.json
    std::ifstream file2(data_path + "/passengers.json");
    json passengers;
    file2 >> passengers;

    // Read from rides.json
    std::ifstream file3(data_path + "/rides.json");
    json rides;
    file3 >> rides;

    // Read from lines.json
    std::ifstream file4(data_path + "/lines.json");
    json lines;
    file4 >> lines;

    // Read from stations.json
    std::ifstream file5(data_path + "/stations.json");
    json stations;
    file5 >> stations;

    // end = std::time(nullptr);   
    gettimeofday(&end, NULL);

    time_spent = (double)(end.tv_sec - start.tv_sec) + ((end.tv_usec - start.tv_usec)/1000000.0);

    std::cout << "JSON parsing completed in " << time_spent << " seconds." << std::endl;

    // Connect to the database
    pqxx::connection c(conn_str);
    pqxx::work w(c);

    // // Create the cards table
    std::string create_cards_query = std::string("CREATE TABLE IF NOT EXISTS cards(")
        +"code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'), "
        +"money MONEY NOT NULL, "
        +"create_time TIMESTAMP NOT NULL);";
    std::cout << create_cards_query << std::endl;
    w.exec(create_cards_query);

    // // Create the passengers table
    std::string create_passengers_query = std::string("CREATE TABLE IF NOT EXISTS passengers (")
        +"name VARCHAR(255) NOT NULL, "
        +"id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'), "
        +"phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'), "
        +"gender VARCHAR(255) NOT NULL, "
        +"district VARCHAR(255) NOT NULL);";
    std::cout << create_passengers_query << std::endl;
    w.exec(create_passengers_query);

    // Create the rides table
    std::string create_rides_query = std::string("CREATE TABLE IF NOT EXISTS rides (")
        +"rail_user VARCHAR(255), "
        +"start_station VARCHAR(255), "
        +"end_station VARCHAR(255), "
        +"price DOUBLE PRECISION, "
        +"start_time TIMESTAMP, "
        +"end_time TIMESTAMP);";
    std::cout << create_rides_query << std::endl;
    w.exec(create_rides_query);
    


    // Create the lines table
    std::string create_lines_query = std::string("CREATE TABLE IF NOT EXISTS lines (")
        // +"line_id VARCHAR(255) , "
        +"name VARCHAR(255)PRIMARY KEY, "
        +"start_time TIME, "
        +"end_time TIME, "
        +"mileage DOUBLE PRECISION, "
        +"color VARCHAR(255), "
        +"first_opening DATE, "
        +"url TEXT,"
        +"intro TEXT);";
    std::cout << create_lines_query << std::endl;
    w.exec(create_lines_query);

    // Create the line_details table
    std::string create_line_details_query = std::string("CREATE TABLE IF NOT EXISTS line_details (")
        +"line_name VARCHAR(255), "
        +"station_name VARCHAR(255)"
        // +"station_id VARCHAR(255),"
        // +" FOREIGN KEY (line_id) REFERENCES lines(line_id), "
        // +"FOREIGN KEY (station_id) REFERENCES stations(station_id)"
        +");";
    std::cout << create_line_details_query << std::endl;
    w.exec(create_line_details_query);

    // Create the stations table
    std::string create_stations_query = std::string("CREATE TABLE IF NOT EXISTS stations (")
        // +"station_id VARCHAR(255) PRIMARY KEY, "
        +"name VARCHAR(255) PRIMARY KEY, "
        +"district VARCHAR(255), "
        +"intro TEXT, "
        +"chinese_name VARCHAR(255));";
    std::cout << create_stations_query << std::endl;
    w.exec(create_stations_query);

    // Create the bus_stations table
    std::string create_bus_stations_query = std::string("CREATE TABLE IF NOT EXISTS bus_stations (")
        // +"station_id VARCHAR(255), "
        +"name VARCHAR(255) PRIMARY KEY, "
        +"district VARCHAR(255));";
    std::cout << create_bus_stations_query << std::endl;
    w.exec(create_bus_stations_query);

    // Create the bus_line table
    std::string create_bus_lines_query = std::string("CREATE TABLE IF NOT EXISTS bus_lines (")
        // +"bus_line_id VARCHAR(255), "
        +"name VARCHAR(255) PRIMARY KEY);";
    std::cout << create_bus_lines_query << std::endl;
    w.exec(create_bus_lines_query);

    // Create the bus_line_details table
    std::string create_bus_line_details_query = std::string("CREATE TABLE IF NOT EXISTS bus_line_details (")
        +"bus_line_name VARCHAR(255), "
        +"bus_station_name VARCHAR(255)"
        // +", FOREIGN KEY (bus_line_name) REFERENCES bus_line(name), "
        // +"FOREIGN KEY (bus_station_name) REFERENCES bus_station(name)"
        +");";
    std::cout << create_bus_line_details_query << std::endl;
    w.exec(create_bus_line_details_query);

    // Create the exits table
    std::string create_exits_query = std::string("CREATE TABLE IF NOT EXISTS exits (")
        // +"exit_id VARCHAR(255), "
        +"station_name VARCHAR(255), "
        +"name VARCHAR(255),"
        +"textt TEXT,"
        +"FOREIGN KEY (station_name) REFERENCES stations(name),"
        +"PRIMARY KEY (station_name, name)"
        // +"PRIMARY KEY exit_id"
        +");";
    std::cout << create_exits_query << std::endl;
    w.exec(create_exits_query);

    std::cout << "Writing to cards.csv..." << std::endl;
    std::filesystem::path cards_csv_path = current_path / "cards.csv";
    std::ofstream cards_csv(cards_csv_path);
    if (!cards_csv.is_open()) {
        std::cerr << "Error: could not open cards.csv" << std::endl;
        return 1;
    }
    
    std::cout << "\tcards.csv path: " << cards_csv_path << std::endl;
    cards_csv << "code,money,create_time" << std::endl;
    for (auto& card : cards) {
        // Extract data from JSON
        std::string code = card["code"];
        double money = card["money"];
        std::string create_time = card["create_time"];
        cards_csv << code << "," << money << "," << create_time << std::endl;
    }
    cards_csv.close();



    std::cout << "Writing to passengers.csv..." << std::endl;
    std::filesystem::path passengers_csv_path = current_path / "passengers.csv";
    std::ofstream passengers_csv(passengers_csv_path);
    if (!passengers_csv.is_open()) {
        std::cerr << "Error: could not open passengers.csv" << std::endl;
        return 1;
    }

    
    std::cout << "\tpassengers.csv path: " << passengers_csv_path << std::endl;
    passengers_csv << "name,id_number,phone_number,gender,district" << std::endl;
    for (auto& passenger : passengers) {
        // Extract data from JSON
        std::string name = passenger["name"];
        std::string id_number = passenger["id_number"];
        std::string phone_number = passenger["phone_number"];
        std::string gender = passenger["gender"];
        std::string district = passenger["district"];
        passengers_csv << name << "," << id_number << "," << phone_number << "," << gender << "," << district << std::endl;
    }
    passengers_csv.close(); 


    
    std::cout << "Writing to rides.csv..." << std::endl;
    std::filesystem::path rides_csv_path = current_path / "rides.csv";
    std::ofstream rides_csv(rides_csv_path);
    if (!rides_csv.is_open()) {
        std::cerr << "Error: could not open rides.csv" << std::endl;
        return 1;
    }

    
    std::cout << "\trides.csv path: " << rides_csv_path << std::endl;

    rides_csv << "rail_user,start_station,end_station,price,start_time,end_time" << std::endl;
    for (auto& ride : rides) {
        // Extract data from JSON
        std::string user = ride["user"];
        std::string start_station = ride["start_station"];
        std::string end_station = ride["end_station"];
        double price = ride["price"];
        std::string start_time = ride["start_time"];
        std::string end_time = ride["end_time"];
        std::string str = user + "," + start_station + "," + end_station + "," + std::to_string(price) + "," + start_time + "," + end_time;
        // str.erase(std::remove(str.begin(), str.end(), '\n'), str.end());
        // rides_csv << str << "\n";
        rides_csv << str << std::endl;

    }
    rides_csv.close();



    std::cout << "Writing to lines.csv..." << std::endl;
    std::filesystem::path lines_csv_path = current_path / "lines.csv";
    std::ofstream lines_csv(lines_csv_path);
    std::cout << "\tlines.csv path: " << lines_csv_path << std::endl;
    lines_csv << "name,start_time,end_time,mileage,color,first_opening,url,intro" << std::endl;



    std::cout << "Writing to line_details..." << std::endl;
    std::filesystem::path line_details_csv_path = current_path / "line_details.csv";
    std::ofstream line_details_csv(line_details_csv_path);
    std::cout << "\tline_details.csv path: " << line_details_csv_path << std::endl;
    line_details_csv << "line_name,station_name" << std::endl;
    // int id;
    // id = 0;
    if (!lines_csv.is_open()||!line_details_csv.is_open()) {
        std::cerr << "Error: could not open csv" << std::endl;
        return 1;
    }

    for (auto& line : lines.items()) {

        std::string name = line.key();
        
        std::string start_time = line.value()["start_time"];
        
        std::string end_time = line.value()["end_time"];
        std::string mileage = line.value()["mileage"];
        std::string color = line.value()["color"];
        std::string first_opening = line.value()["first_opening"];
        std::string url = line.value()["url"];
        std::string intro = line.value()["intro"];
        std::string line_str = name + "," + start_time + "," + end_time + "," + mileage + "," + color + "," + first_opening + "," + url + "," + intro;
        line_str.erase(std::remove(line_str.begin(), line_str.end(), '\n'), line_str.end());
        lines_csv << line_str << std::endl;
        
        for (const auto& station : line.value()["stations"]) {
            // std::cout << "  Station: " << station.get<std::string>() << std::endl;
            std::string line_details_str = name + "," + station.get<std::string>();
            
            line_details_str.erase(std::remove(line_details_str.begin(), line_details_str.end(), '\n'), line_details_str.end());
            line_details_csv << line_details_str << std::endl;
        }
    }
    lines_csv.close();
    line_details_csv.close();



    std::cout << "Writing to stations.csv..." << std::endl;
    std::filesystem::path stations_csv_path = current_path / "stations.csv";
    std::ofstream stations_csv(stations_csv_path);
    std::cout << "\tstations.csv path: " << stations_csv_path << std::endl;
    stations_csv << "name,district,intro,chinese_name"<< std::endl;

    std::cout << "Writing to bus_stations.csv..." << std::endl;
    std::filesystem::path bus_stations_csv_path = current_path / "bus_stations.csv";
    std::ofstream bus_stations_csv(bus_stations_csv_path);
    std::cout << "\tbus_stations.csv path: " << bus_stations_csv_path << std::endl;
    bus_stations_csv << "name,district" << std::endl;

    std::cout << "Writing to exits.csv..." << std::endl;
    std::filesystem::path exits_csv_path = current_path / "exits.csv";
    std::ofstream exits_csv(exits_csv_path);
    std::cout << "\texits.csv path: " << exits_csv_path << std::endl;
    exits_csv << "station_name,name,textt" << std::endl;


    std::cout << "Writing to bus_line_details.csv..." << std::endl;
    std::filesystem::path bus_line_details_csv_path = current_path / "bus_line_details.csv";
    std::ofstream bus_line_details_csv(bus_line_details_csv_path);
    std::cout << "\tbus_line_details.csv path: " << bus_line_details_csv_path << std::endl;
    bus_line_details_csv << "bus_line_name,station_name" << std::endl;    

    if (!bus_line_details_csv.is_open()||!exits_csv.is_open()||!stations_csv.is_open()||!bus_stations_csv.is_open()){
        std::cerr << "Error: could not open csv" << std::endl;
        return 1;
    }
    
    for (auto& station : stations.items()) {

        std::string name = station.key();
        std::string district = station.value()["district"];
        std::string intro = station.value()["intro"];
        // Replace commas with Chinese commas, using wstring because the Chinese characters are longer than 1 byte
        std::wstring intro_wstr = converter.from_bytes(intro);
        std::replace(intro_wstr.begin(), intro_wstr.end(), L',', L'，');
        intro = converter.to_bytes(intro_wstr);

        std::string chinese_name = station.value()["chinese_name"];
        std::string station_str = name + "," + district + "," + intro + "," + chinese_name;
        station_str.erase(std::remove(station_str.begin(), station_str.end(), '\n'), station_str.end());
        stations_csv << station_str << std::endl;

        for (const auto& exit : station.value()["out_info"]) {
            if (!exit.contains("textt") && exit["textt"].empty()) {
                continue;
            }
            std::string exit_name = exit["outt"];


            std::string exit_text = exit["textt"];
            std::wstring exit_text_wstr = converter.from_bytes(exit_text);
            std::replace(exit_text_wstr.begin(), exit_text_wstr.end(), L',', L'、');
            std::replace(exit_text_wstr.begin(), exit_text_wstr.end(), L'，', L'、');
            exit_text = converter.to_bytes(exit_text_wstr);

            std::string exit_str = name + "," + exit_name + "," + exit_text;
            exit_str.erase(std::remove(exit_str.begin() + name.size() - 1, exit_str.end(), ' '), exit_str.end());
            exits_csv << exit_str << std::endl;
        }
        

        for (const auto& bus_station : station.value()["bus_info"]) {
            if (!bus_station.contains("busOutInfo") || bus_station["busOutInfo"].empty()) {
                continue;// skip stations without bus info
            }
            std::string bus_station_name = bus_station["busOutInfo"][0]["busName"];
            if (bus_station_name.empty()) {// skip empty bus station names
                continue;
            }
            // std::cout << bus_station_name << std::endl; 
            std::string bus_station_str = bus_station_name + "," + district;
            bus_station_str.erase(std::remove(bus_station_str.begin(), bus_station_str.end(), '\n'), bus_station_str.end());
            bus_stations_csv << bus_station_str << std::endl;

            // add bus line details for each bus station
            std::string bus_line_name = bus_station["busOutInfo"][0]["busInfo"].get<std::string>();
            std::wstring bus_line_details = converter.from_bytes(bus_line_name);
            size_t pos;

            while ((pos = bus_line_details.find_first_of(L",，路")) != std::wstring::npos) {
                if (bus_line_details[pos] == L'路') {
                    bus_line_details.replace(pos, 1, L" ");
                } else {
                    bus_line_details.replace(pos, 1, L"、");
                }
            }


            std::wstringstream wss(bus_line_details);
            std::wstring token;
            std::vector<std::string> bus_line_details_arr;
            std::wstring_convert<std::codecvt_utf8<wchar_t>, wchar_t> converter;

            while (std::getline(wss, token, L'、')) {
                if (!token.empty()) {\
                    bus_line_details_arr.push_back(converter.to_bytes(token));
                }
            }
            std::string bus_line_str;
            for (const auto& bus_line : bus_line_details_arr) {
                // bus_line_str = bus_line_str + bus_line + "," + bus_station_name + "\n";
                bus_line_str = bus_line + "," + bus_station_name;
                // bus_line_str.erase(std::remove(bus_line_str.begin(), bus_line_str.end(), '\n'), bus_line_str.end());
                bus_line_str.erase(std::remove(bus_line_str.begin(), bus_line_str.end(), ' '), bus_line_str.end());
                bus_line_details_csv << bus_line_str << std::endl;
            }

        }



    }
    stations_csv.close();
    exits_csv.close();
    bus_stations_csv.close();
    bus_line_details_csv.close();

    gettimeofday(&end, NULL);
    time_spent = (double)(end.tv_sec - start.tv_sec) + ((end.tv_usec - start.tv_usec)/1000000.0);
    std::cout << "json to csv completed in " << time_spent << " seconds." << std::endl;
    gettimeofday(&start, NULL);
    std::string copy_cards_query = "COPY cards FROM '" + cards_csv_path.string() + "' WITH (FORMAT CSV, HEADER, DELIMITER ',')";
    std::cout << copy_cards_query << std::endl;
    w.exec(copy_cards_query);

    std::string copy_passengers_query = "COPY passengers FROM '" + passengers_csv_path.string() + "' WITH (FORMAT CSV, HEADER, DELIMITER ',')";
    std::cout << copy_passengers_query << std::endl;
    w.exec(copy_passengers_query);

    std::string copy_rides_query = "COPY rides FROM '" + rides_csv_path.string() + "' WITH (FORMAT CSV, HEADER, DELIMITER ',')";
    std::cout << copy_rides_query << std::endl;
    w.exec(copy_rides_query);

    std::string copy_lines_query = "COPY lines FROM '" + lines_csv_path.string() + "' WITH (FORMAT CSV, HEADER, DELIMITER ',')";
    std::cout << copy_lines_query << std::endl;
    w.exec(copy_lines_query);

    std::string copy_line_details_query = "COPY line_details FROM '" + line_details_csv_path.string() + "' WITH (FORMAT CSV, HEADER, DELIMITER ',')";
    std::cout << copy_line_details_query << std::endl;
    w.exec(copy_line_details_query);

    std::string copy_stations_query = "COPY stations FROM '" + stations_csv_path.string() + "' WITH (FORMAT CSV, HEADER, DELIMITER ',')";
    std::cout << copy_stations_query << std::endl;
    w.exec(copy_stations_query);

    std::string copy_exits_query = "COPY exits FROM '" + exits_csv_path.string() + "' WITH (FORMAT CSV, HEADER, DELIMITER ',')";
    std::cout << copy_exits_query << std::endl;
    w.exec(copy_exits_query);
    
    std::string copy_bus_stations_query = std::string("CREATE TEMP TABLE tmp_bus_stations AS SELECT * FROM bus_stations LIMIT 0;")
    +"COPY tmp_bus_stations FROM '"
    + bus_stations_csv_path.string()
    +"' WITH (FORMAT CSV, HEADER, DELIMITER ',');"
    +"INSERT INTO bus_stations SELECT * FROM tmp_bus_stations ON CONFLICT DO NOTHING;";
    std::cout << copy_bus_stations_query << std::endl;
    w.exec(copy_bus_stations_query);

    std::string copy_bus_line_details_query = "COPY bus_line_details FROM '" + bus_line_details_csv_path.string() + "' WITH (FORMAT CSV, HEADER, DELIMITER ',')";
    std::cout << copy_bus_line_details_query << std::endl;
    w.exec(copy_bus_line_details_query);

    std::string make_bus_line_query = "INSERT INTO bus_lines SELECT DISTINCT bus_line_name FROM bus_line_details;";
    std::cout << make_bus_line_query << std::endl;
    w.exec(make_bus_line_query);
    w.commit();

    gettimeofday(&end, NULL);
    time_spent = (double)(end.tv_sec - start.tv_sec) + ((end.tv_usec - start.tv_usec)/1000000.0);
    std::cout << "csv to database completed in " << time_spent << " seconds." << std::endl;

    return 0;
}

SyntaxError: invalid syntax (1421186486.py, line 1)

In [2]:
import json
import os
import psycopg
from psycopg import sql
import csv
import codecs
import time
import datetime
import shutil

# def printProgress(percentage):
#     val = percentage * 100
#     lpad = int(percentage * PBWIDTH)
#     rpad = PBWIDTH - lpad
#     print("\r%4.1f%% [%.*s%*s]" % (val, lpad, PBSTR, rpad, ""), end="")
#     sys.stdout.flush()


csv_path_ctsr = os.getenv("CSV_PATH")
csv_path = csv_path_ctsr if csv_path_ctsr else "?"

current_path = os.path.join(csv_path)

# Chinese string
chu1_ru4_kou3 = "出入口"

# timing
start = time.time()

print("Parsing JSON files...")

# Read from cards.json
with open( "../data/cards.json") as file:
    cards = json.load(file)

# Read from passengers.json
with open("../data/passengers.json") as file:
    passengers = json.load(file)

# Read from rides.json
with open("../data/rides.json") as file:
    rides = json.load(file)

# Read from lines.json
with open("../data/lines.json") as file:
    lines = json.load(file)

# Read from stations.json
with open("../data/stations.json") as file:
    stations = json.load(file)

end = time.time()

time_spent = end - start

print("JSON parsing completed in", time_spent, "seconds.")

db_name = 'project1'
db_user = 'superlucky'
db_password = ''
db_host = "localhost"
db_port = 5432
print(db_name, db_user, db_password, db_host, db_port,'\n')

# Create a connection to the database
conn = psycopg.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)

cur = conn.cursor()


# Create the cards table
create_cards_query = "CREATE TABLE IF NOT EXISTS cards(code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'), money MONEY NOT NULL, create_time TIMESTAMP NOT NULL);"
print(create_cards_query)
cur.execute(create_cards_query)

# Create the passengers table
create_passengers_query = "CREATE TABLE IF NOT EXISTS passengers (name VARCHAR(255) NOT NULL, id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'), phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'), gender VARCHAR(255) NOT NULL, district VARCHAR(255) NOT NULL);"
print(create_passengers_query)
cur.execute(create_passengers_query)

# Create the rides table
create_rides_query = "CREATE TABLE IF NOT EXISTS rides (rail_user VARCHAR(255), start_station VARCHAR(255), end_station VARCHAR(255), price DOUBLE PRECISION, start_time TIMESTAMP, end_time TIMESTAMP);"
print(create_rides_query)
cur.execute(create_rides_query)

# Create the lines table
create_lines_query = "CREATE TABLE IF NOT EXISTS lines (name VARCHAR(255) PRIMARY KEY, start_time TIME, end_time TIME, mileage DOUBLE PRECISION, color VARCHAR(255), first_opening DATE, url TEXT, intro TEXT);"
print(create_lines_query)
cur.execute(create_lines_query)

# Create the line_details table
create_line_details_query = "CREATE TABLE IF NOT EXISTS line_details (line_name VARCHAR(255), station_name VARCHAR(255));"
print(create_line_details_query)
cur.execute(create_line_details_query)

# Create the stations table
create_stations_query = "CREATE TABLE IF NOT EXISTS stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255), intro TEXT, chinese_name VARCHAR(255));"
print(create_stations_query)
cur.execute(create_stations_query)

# Create the bus_stations table
create_bus_stations_query = "CREATE TABLE IF NOT EXISTS bus_stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255));"
print(create_bus_stations_query)
cur.execute(create_bus_stations_query)

# Create the bus_lines table
create_bus_lines_query = "CREATE TABLE IF NOT EXISTS bus_lines (name VARCHAR(255) PRIMARY KEY);"
print(create_bus_lines_query)
cur.execute(create_bus_lines_query)

# Create the bus_line_details table
create_bus_line_details_query = "CREATE TABLE IF NOT EXISTS bus_line_details (bus_line_name VARCHAR(255), bus_station_name VARCHAR(255));"
print(create_bus_line_details_query)
cur.execute(create_bus_line_details_query)

# Create the exits table
create_exits_query = "CREATE TABLE IF NOT EXISTS exits (station_name VARCHAR(255), name VARCHAR(255), textt TEXT, PRIMARY KEY (station_name, name));"
print(create_exits_query)
cur.execute(create_exits_query)

# Write to cards.csv
cards_csv_path = "../Data/cards.csv"
with open(cards_csv_path, mode='w', newline='') as cards_csv:
    writer = csv.writer(cards_csv)
    writer.writerow(["code", "money", "create_time"])
    for card in cards:
        code = card["code"]
        money = card["money"]
        create_time = card["create_time"]
        writer.writerow([code, money, create_time])

# Write to passengers.csv
passengers_csv_path = "../Data/passengers.csv"
with open(passengers_csv_path, mode='w', newline='') as passengers_csv:
    writer = csv.writer(passengers_csv)
    writer.writerow(["name", "id_number", "phone_number", "gender", "district"])
    for passenger in passengers:
        name = passenger["name"]
        id_number = passenger["id_number"]
        phone_number = passenger["phone_number"]
        gender = passenger["gender"]
        district = passenger["district"]
        writer.writerow([name, id_number, phone_number, gender, district])

# Write to rides.csv
rides_csv_path = "../Data/rides.csv"
with open(rides_csv_path, mode='w', newline='') as rides_csv:
    writer = csv.writer(rides_csv)
    writer.writerow(["rail_user", "start_station", "end_station", "price", "start_time", "end_time"])
    for ride in rides:
        rail_user = ride["user"]
        start_station = ride["start_station"]
        end_station = ride["end_station"]
        price = ride["price"]
        start_time = ride["start_time"]
        end_time = ride["end_time"]
        writer.writerow([rail_user, start_station, end_station, price, start_time, end_time])

# Write to lines.csv
lines_csv_path = "../Data/lines.csv"
with open(lines_csv_path, mode='w', newline='') as lines_csv:
    writer = csv.writer(lines_csv)
    writer.writerow(["name", "start_time", "end_time", "mileage", "color", "first_opening", "url", "intro"])
    for line_name, line in lines.items():
        name = line_name
        start_time = line["start_time"]
        end_time = line["end_time"]
        mileage = line["mileage"]
        color = line["color"]
        first_opening = line["first_opening"]
        url = line["url"]
        intro = line["intro"]
        writer.writerow([name, start_time, end_time, mileage, color, first_opening, url, intro])

# Write to line_details.csv
line_details_csv_path = "../Data/line_details.csv"
with open(line_details_csv_path, mode='w', newline='') as line_details_csv:
    writer = csv.writer(line_details_csv)
    writer.writerow(["line_name", "station_name"])
    for line_name, line in lines.items():
        for station in line["stations"]:
            line_name = line_name
            station_name = station
            writer.writerow([line_name, station_name])

# Write to stations.csv
stations_csv_path = "../Data/stations.csv"
with open(stations_csv_path, mode='w', newline='') as stations_csv:
    writer = csv.writer(stations_csv)
    writer.writerow(["name", "district", "intro", "chinese_name"])
    for station_name, station in stations.items():
        name = station_name
        district = station["district"]
        intro = station["intro"]
        chinese_name = station["chinese_name"]
        writer.writerow([name, district, intro, chinese_name])

print(stations.items())
# Write to bus_stations.csv
bus_stations_csv_path = "../Data/bus_stations.csv"
with open(bus_stations_csv_path, mode='w', newline='') as bus_stations_csv:
    writer = csv.writer(bus_stations_csv)
    writer.writerow(["name", "district"])
    for station_name, station in stations.items():
        if "bus_info" in station:
            for bus_station in station["bus_info"]:
                if "busOutInfo" in bus_station and bus_station["busOutInfo"]:
                    bus_station_name = bus_station["busOutInfo"][0]["busName"]
                    district = station["district"]
                    writer.writerow([bus_station_name, district])

# Write to exits.csv
exits_csv_path = "../Data/exits.csv"
with open(exits_csv_path, mode='w', newline='') as exits_csv:
    writer = csv.writer(exits_csv)
    writer.writerow(["station_name", "name", "textt"])
    for station_name, station in stations.items():
        for exit_info in station["out_info"]:
            if "textt" in exit_info and exit_info["textt"]:
                station_name = station_name
                name = exit_info["outt"]
                textt = exit_info["textt"]
                writer.writerow([station_name, name, textt])

# Write to bus_line_details.csv
bus_line_details_csv_path = "../Data/bus_line_details.csv"
with open(bus_line_details_csv_path, mode='w', newline='') as bus_line_details_csv:
    writer = csv.writer(bus_line_details_csv)
    writer.writerow(["bus_line_name", "station_name"])
    for station_name, station in stations.items():
        if "bus_info" in station:
            for bus_station in station["bus_info"]:
                if "busOutInfo" in bus_station and bus_station["busOutInfo"]:
                    bus_station_name = bus_station["busOutInfo"][0]["busName"]
                bus_line_name = bus_station["busOutInfo"][0]["busInfo"]
                writer.writerow([bus_line_name, bus_station_name])

# Copy the csv files to the database
copy_cards_query = sql.SQL("COPY cards FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')")
cur.execute(copy_cards_query, [cards_csv_path])

copy_passengers_query = sql.SQL("COPY passengers FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')")
cur.execute(copy_passengers_query, [passengers_csv_path])

copy_rides_query = sql.SQL("COPY rides FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')")
cur.execute(copy_rides_query, [rides_csv_path])

copy_lines_query = sql.SQL("COPY lines FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')")
cur.execute(copy_lines_query, [lines_csv_path])

copy_line_details_query = sql.SQL("COPY line_details FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')")
cur.execute(copy_line_details_query, [line_details_csv_path])

copy_stations_query = sql.SQL("COPY stations FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')")
cur.execute(copy_stations_query, [stations_csv_path])

copy_exits_query = sql.SQL("COPY exits FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')")
cur.execute(copy_exits_query, [exits_csv_path])

copy_bus_stations_query = sql.SQL("CREATE TEMP TABLE tmp_bus_stations AS SELECT * FROM bus_stations LIMIT 0;")
+ sql.SQL("COPY tmp_bus_stations FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',');")
+ sql.SQL("INSERT INTO bus_stations SELECT * FROM tmp_bus_stations ON CONFLICT DO NOTHING;")
cur.execute(copy_bus_stations_query, [bus_stations_csv_path])

copy_bus_line_details_query = sql.SQL("COPY bus_line_details FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')")
cur.execute(copy_bus_line_details_query, [bus_line_details_csv_path])

make_bus_line_query = sql.SQL("INSERT INTO bus_lines SELECT DISTINCT bus_line_name FROM bus_line_details;")
cur.execute(make_bus_line_query)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()




Parsing JSON files...
JSON parsing completed in 0.18164777755737305 seconds.
project1 superlucky  localhost 5432 

CREATE TABLE IF NOT EXISTS cards(code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'), money MONEY NOT NULL, create_time TIMESTAMP NOT NULL);
CREATE TABLE IF NOT EXISTS passengers (name VARCHAR(255) NOT NULL, id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'), phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'), gender VARCHAR(255) NOT NULL, district VARCHAR(255) NOT NULL);
CREATE TABLE IF NOT EXISTS rides (rail_user VARCHAR(255), start_station VARCHAR(255), end_station VARCHAR(255), price DOUBLE PRECISION, start_time TIMESTAMP, end_time TIMESTAMP);
CREATE TABLE IF NOT EXISTS lines (name VARCHAR(255) PRIMARY KEY, start_time TIME, end_time TIME, mileage DOUBLE PRECISION, color VARCHAR(255), first_opening DATE, url TEXT, intro TEXT);
CREATE TABLE IF NOT EXISTS line_details (line_name VARCHAR(255), station_name VARCHAR(25

IndexError: list index out of range

In [1]:
import json
import os
import psycopg
from psycopg import sql
import csv
import time

def load_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)

def setup_database(cursor):
    create_table_commands = [
        """
        CREATE TABLE IF NOT EXISTS cards (
            code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'),
            money MONEY NOT NULL,
            create_time TIMESTAMP NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS passengers (
            name VARCHAR(255) NOT NULL,
            id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'),
            phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'),
            gender VARCHAR(255) NOT NULL,
            district VARCHAR(255) NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS rides (
            rail_user VARCHAR(255) NOT NULL,
            start_station VARCHAR(255) NOT NULL,
            end_station VARCHAR(255) NOT NULL,
            price DOUBLE PRECISION NOT NULL,
            start_time TIMESTAMP NOT NULL,
            end_time TIMESTAMP NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS lines (
            name VARCHAR(255) PRIMARY KEY,
            start_time TIME NOT NULL,
            end_time TIME NOT NULL,
            mileage DOUBLE PRECISION NOT NULL,
            color VARCHAR(255) NOT NULL,
            first_opening DATE NOT NULL,
            url TEXT,
            intro TEXT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS line_details (
            line_name VARCHAR(255) NOT NULL,
            station_name VARCHAR(255) NOT NULL,
            PRIMARY KEY (line_name, station_name)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS stations (
            name VARCHAR(255) PRIMARY KEY,
            district VARCHAR(255) NOT NULL,
            intro TEXT,
            chinese_name VARCHAR(255) NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS bus_stations (
            name VARCHAR(255) PRIMARY KEY,
            district VARCHAR(255) NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS bus_lines (
            name VARCHAR(255) PRIMARY KEY
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS bus_line_details (
            bus_line_name VARCHAR(255) NOT NULL,
            bus_station_name VARCHAR(255) NOT NULL,
            PRIMARY KEY (bus_line_name, bus_station_name)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS exits (
            station_name VARCHAR(255) NOT NULL,
            name VARCHAR(255) NOT NULL,
            textt TEXT,
            PRIMARY KEY (station_name, name)
        );
        """
    ]
    for command in create_table_commands:
        cursor.execute(command)

def write_to_csv_and_db(data, headers, filename, table_name, cursor, db_conn):
    csv_path = os.path.join(os.getenv("CSV_PATH", "."), filename)
    with open(csv_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(data)

    copy_sql = sql.SQL("COPY {} FROM %s WITH (FORMAT csv, HEADER)").format(sql.Identifier(table_name))
    cursor.execute(copy_sql, [csv_path])
    db_conn.commit()

def main():
    # Connect to the PostgreSQL database
    conn = psycopg.connect(dbname="project1", user="superlucky", password="", host="localhost", port="5432")
    cur = conn.cursor()
    
    # Setup database: create tables
    setup_database(cur)

    # Loading JSON data
    json_files = ["cards.json", "passengers.json", "rides.json", "lines.json", "stations.json"]
    json_data = {file.split('.')[0]: load_json(os.path.join("../data", file)) for file in json_files}

    # Preparing data for CSV files and copying to the database
    csv_info = {
        "cards": (["code", "money", "create_time"], [("code", "money", "create_time")]),
        "passengers": (["name", "id_number", "phone_number", "gender", "district"], [("name", "id_number", "phone_number", "gender", "district")]),
        "rides": (["rail_user", "start_station", "end_station", "price", "start_time", "end_time"], [("user", "start_station", "end_station", "price", "start_time", "end_time")]),
        "lines": (["name", "start_time", "end_time", "mileage", "color", "first_opening", "url", "intro"], [("name", "start_time", "end_time", "mileage", "color", "first_opening", "url", "intro")]),
        "stations": (["name", "district", "intro", "chinese_name"], [("name", "district", "intro", "chinese_name")])
    }

    for key, (headers, fields) in csv_info.items():
        data = [
            [item[field] for field in fields] for item in json_data[key]
        ]
        write_to_csv_and_db(data, headers, f"{key}.csv", key, cur, conn)

    # Cleanup
    cur.close()
    conn.close()

if __name__ == "__main__":
    main()


KeyError: ('code', 'money', 'create_time')

In [27]:
import json
import os
import psycopg
from psycopg import sql
import csv
import time
from concurrent.futures import ThreadPoolExecutor
import sys

def load_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)

def setup_database(cursor):
    create_table_commands = [
        """
        CREATE TABLE IF NOT EXISTS cards (
            code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'),
            money MONEY NOT NULL,
            create_time TIMESTAMP NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS passengers (
            name VARCHAR(255) NOT NULL,
            id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'),
            phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'),
            gender VARCHAR(255) NOT NULL,
            district VARCHAR(255) NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS rides (
            rail_user VARCHAR(255) NOT NULL,
            start_station VARCHAR(255) NOT NULL,
            end_station VARCHAR(255) NOT NULL,
            price DOUBLE PRECISION NOT NULL,
            start_time TIMESTAMP NOT NULL,
            end_time TIMESTAMP NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS lines (
            name VARCHAR(255) PRIMARY KEY,
            start_time TIME NOT NULL,
            end_time TIME NOT NULL,
            mileage DOUBLE PRECISION NOT NULL,
            color VARCHAR(255) NOT NULL,
            first_opening DATE NOT NULL,
            url TEXT,
            intro TEXT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS line_details (
            line_name VARCHAR(255) NOT NULL,
            station_name VARCHAR(255) NOT NULL,
            PRIMARY KEY (line_name, station_name)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS stations (
            name VARCHAR(255) PRIMARY KEY,
            district VARCHAR(255) NOT NULL,
            intro TEXT,
            chinese_name VARCHAR(255) NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS bus_stations (
            name VARCHAR(255) PRIMARY KEY,
            district VARCHAR(255) NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS bus_lines (
            name VARCHAR(255) PRIMARY KEY
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS bus_line_details (
            bus_line_name VARCHAR(255) NOT NULL,
            bus_station_name VARCHAR(255) NOT NULL,
            PRIMARY KEY (bus_line_name, bus_station_name)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS exits (
            station_name VARCHAR(255) NOT NULL,
            name VARCHAR(255) NOT NULL,
            textt TEXT,
            PRIMARY KEY (station_name, name)
        );
        """
    ]
    for command in create_table_commands:
        cursor.execute(command)

def write_to_csv_and_db(data, headers, filename, table_name, cursor, db_conn):
    csv_path = os.path.join(os.getenv("CSV_PATH", "."), filename)
    with open(csv_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(data)

    copy_sql = sql.SQL("COPY {} FROM %s WITH (FORMAT csv, HEADER)").format(sql.Identifier(table_name))
    cursor.execute(copy_sql, [csv_path])
    db_conn.commit()
    print(f"Data for {table_name} written to {csv_path} and copied to database.")

def process_file(json_file, db_conn, cur):
    print(f"Processing {json_file}...")
    start = time.time()
    data = load_json(os.path.join("../data", json_file))
    entity = json_file.replace('.json', '')
    headers = {
        'cards': ["code", "money", "create_time"],
        'passengers': ["name", "id_number", "phone_number", "gender", "district"],
        'rides': ["rail_user", "start_station", "end_station", "price", "start_time", "end_time"],
        'lines': ["name", "start_time", "end_time", "mileage", "color", "first_opening", "url", "intro"],
        'stations': ["name", "district", "intro", "chinese_name"]
    }[entity]

    rows = [
        [item[key] for key in headers] for item in data
    ]

    write_to_csv_and_db(rows, headers, f"{entity}.csv", entity, cur, db_conn)
    print(f"Completed {json_file} in {time.time() - start:.2f} seconds.")

def main():
    conn = psycopg.connect(dbname="project1", user="superlucky", password="", host="localhost", port="5432")
    cur = conn.cursor()

    setup_database(cur)

    json_files = ["cards.json", "passengers.json", "rides.json", "lines.json", "stations.json"]
    
    with ThreadPoolExecutor(max_workers=len(json_files)) as executor:
        futures = [executor.submit(process_file, json_file, conn, cur) for json_file in json_files]
        for future in futures:
            future.result()  # Wait for all futures to complete

    cur.close()
    conn.close()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [1]:
import json
import os
import psycopg
from psycopg import sql
import csv
from multiprocessing import Pool, cpu_count

def load_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)

def execute_sql_commands(cursor, commands):
    for command in commands:
        cursor.execute(command)

def write_to_csv(data, headers, filename):
    csv_path = os.path.join("/tmp", filename)
    with open(csv_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(data)
    return csv_path

def process_json_data(json_file, cursor, db_conn):
    print(f"Processing {json_file}")
    data = load_json(os.path.join("../data", json_file))
    entity = json_file.replace('.json', '')
    headers = {
        'cards': ["code", "money", "create_time"],
        'passengers': ["name", "id_number", "phone_number", "gender", "district"],
        'rides': ["rail_user", "start_station", "end_station", "price", "start_time", "end_time"],
        'lines': ["name", "start_time", "end_time", "mileage", "color", "first_opening", "url", "intro"],
        'line_details': ["line_name", "station_name"],
        'stations': ["name", "district", "intro", "chinese_name"],
        'exits': ["station_name", "name", "textt"],
        'bus_stations': ["name", "district"],
        'bus_line_details': ["bus_line_name", "bus_station_name"]
    }[entity]

    formatted_data = [[item[field] for field in headers] for item in data]
    csv_path = write_to_csv(formatted_data, headers, f"{entity}.csv")

    copy_command = sql.SQL("COPY {} FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')").format(sql.Identifier(entity))
    cursor.execute(copy_command, [csv_path])
    db_conn.commit()
    print(f"Data for {entity} copied to database.")

def main():
    conn = psycopg.connect(dbname="project1", user="superlucky", password="", host="localhost", port="5432")
    cur = conn.cursor()

    # SQL commands to setup and clean the database
    setup_commands = [
        "DROP TABLE IF EXISTS rides CASCADE;",
        "DROP TABLE IF EXISTS cards CASCADE;",
        "DROP TABLE IF EXISTS passengers CASCADE;",
        "DROP TABLE IF EXISTS line_details CASCADE;",
        "DROP TABLE IF EXISTS lines CASCADE;",
        "DROP TABLE IF EXISTS exits CASCADE;",
        "DROP TABLE IF EXISTS bus_stations CASCADE;",
        "DROP TABLE IF EXISTS stations CASCADE;",
        "DROP TABLE IF EXISTS bus_line_details CASCADE;",
        "DROP TABLE IF EXISTS bus_lines CASCADE;",

        "CREATE TABLE IF NOT EXISTS cards(code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'), money MONEY NOT NULL, create_time TIMESTAMP NOT NULL);",
        "CREATE TABLE IF NOT EXISTS passengers (name VARCHAR(255) NOT NULL, id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'), phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'), gender VARCHAR(255) NOT NULL, district VARCHAR(255) NOT NULL);",
        "CREATE TABLE IF NOT EXISTS rides (rail_user VARCHAR(255), start_station VARCHAR(255), end_station VARCHAR(255), price DOUBLE PRECISION, start_time TIMESTAMP, end_time TIMESTAMP);",
        "CREATE TABLE IF NOT EXISTS lines (name VARCHAR(255) PRIMARY KEY, start_time TIME, end_time TIME, mileage DOUBLE PRECISION, color VARCHAR(255), first_opening DATE, url TEXT, intro TEXT);",
        "CREATE TABLE IF NOT EXISTS line_details (line_name VARCHAR(255), station_name VARCHAR(255));",
        "CREATE TABLE IF NOT EXISTS stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255), intro TEXT, chinese_name VARCHAR(255));",
        "CREATE TABLE IF NOT EXISTS bus_stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255));",
        "CREATE TABLE IF NOT EXISTS bus_line (name VARCHAR(255) PRIMARY KEY);",
        "CREATE TABLE IF NOT EXISTS bus_line_details (bus_line_name VARCHAR(255), bus_station_name VARCHAR(255), FOREIGN KEY (bus_line_name) REFERENCES bus_line(name), FOREIGN KEY (bus_station_name) REFERENCES bus_stations(name));",
        "CREATE TABLE IF NOT EXISTS exits (station_name VARCHAR(255), name VARCHAR(255), textt TEXT, FOREIGN KEY (station_name) REFERENCES stations(name), PRIMARY KEY (station_name, name));"
    ]
    execute_sql_commands(cur, setup_commands)

    json_files = ["cards.json", "passengers.json", "rides.json", "lines.json", "line_details.json", "stations.json", "exits.json", "bus_stations.json", "bus_line_details.json"]
    for json_file in json_files:
        process_json_data(json_file, cur, conn)

    cur.close()
    conn.close()
    print("All data processed and uploaded successfully.")

if __name__ == "__main__":
    main()


Processing cards.json


SyntaxError: syntax error at or near "$1"
LINE 1: COPY "cards" FROM $1 WITH (FORMAT CSV, HEADER, DELIMITER ','...
                          ^

In [1]:
import json
import os
import psycopg
from psycopg import sql
import csv

def load_json(filepath):
    """ Load JSON data from filepath """
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)

def write_to_csv(data, headers, filename):
    """ Write data to CSV file located at /tmp/ directory """
    csv_path = os.path.join("/tmp", filename)
    with open(csv_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(data)
    return csv_path

def process_json_data(json_file):
    """ Process JSON file and return data formatted for CSV """
    data = load_json(os.path.join("../data", json_file))
    entity = json_file.replace('.json', '')
    headers = {
        'cards': ["code", "money", "create_time"],
        'passengers': ["name", "id_number", "phone_number", "gender", "district"],
        'rides': ["rail_user", "start_station", "end_station", "price", "start_time", "end_time"],
        'lines': ["name", "start_time", "end_time", "mileage", "color", "first_opening", "url", "intro"],
        'line_details': ["line_name", "station_name"],
        'stations': ["name", "district", "intro", "chinese_name"],
        'exits': ["station_name", "name", "textt"],
        'bus_stations': ["name", "district"],
        'bus_line_details': ["bus_line_name", "bus_station_name"]
    }[entity]
    formatted_data = [[item[field] for field in headers] for item in data]
    csv_path = write_to_csv(formatted_data, headers, f"{entity}.csv")
    return entity, csv_path

def main():
    conn = psycopg.connect(dbname="project1", user="superlucky", password="", host="localhost", port="5432")
    cur = conn.cursor()

    # SQL commands to clean up and set up the database
    cur.execute("DROP TABLE IF EXISTS rides, cards, passengers, line_details, lines, exits, bus_stations, stations, bus_line_details, bus_lines CASCADE;")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cards(code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'), money MONEY NOT NULL, create_time TIMESTAMP NOT NULL);
        CREATE TABLE IF NOT EXISTS passengers (name VARCHAR(255) NOT NULL, id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'), phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'), gender VARCHAR(255) NOT NULL, district VARCHAR(255) NOT NULL);
        CREATE TABLE IF NOT EXISTS rides (rail_user VARCHAR(255), start_station VARCHAR(255), end_station VARCHAR(255), price DOUBLE PRECISION, start_time TIMESTAMP, end_time TIMESTAMP);
        CREATE TABLE IF NOT EXISTS lines (name VARCHAR(255) PRIMARY KEY, start_time TIME, end_time TIME, mileage DOUBLE PRECISION, color VARCHAR(255), first_opening DATE, url TEXT, intro TEXT);
        CREATE TABLE IF NOT EXISTS line_details (line_name VARCHAR(255), station_name VARCHAR(255));
        CREATE TABLE IF NOT EXISTS stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255), intro TEXT, chinese_name VARCHAR(255));
        CREATE TABLE IF NOT EXISTS bus_stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255));
        CREATE TABLE IF NOT EXISTS bus_lines (name VARCHAR(255) PRIMARY KEY);
        CREATE TABLE IF NOT EXISTS bus_line_details (bus_line_name VARCHAR(255), bus_station_name VARCHAR(255), FOREIGN KEY (bus_line_name) REFERENCES bus_lines(name), FOREIGN KEY (bus_station_name) REFERENCES bus_stations(name));
        CREATE TABLE IF NOT EXISTS exits (station_name VARCHAR(255), name VARCHAR(255), textt TEXT, FOREIGN KEY (station_name) REFERENCES stations(name), PRIMARY KEY (station_name, name));
    """)

    json_files = ["cards.json", "passengers.json", "rides.json", "lines.json", "line_details.json", "stations.json", "exits.json", "bus_stations.json", "bus_line_details.json"]
    for json_file in json_files:
        entity, csv_path = process_json_data(json_file)
        cur.execute(sql.SQL("COPY {} FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')").format(sql.Identifier(entity)), [csv_path])

    cur.execute("""
        CREATE TEMP TABLE tmp_bus_stations AS SELECT * FROM bus_stations LIMIT 0;
        COPY tmp_bus_stations FROM '/tmp/bus_stations.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
        INSERT INTO bus_stations SELECT * FROM tmp_bus_stations ON CONFLICT DO NOTHING;
        INSERT INTO bus_lines SELECT DISTINCT bus_line_name FROM bus_line_details;
    """)
    
    conn.commit()
    cur.close()
    conn.close()
    print("All data processed and uploaded successfully.")

if __name__ == "__main__":
    main()


SyntaxError: syntax error at or near "$1"
LINE 1: COPY "cards" FROM $1 WITH (FORMAT CSV, HEADER, DELIMITER ','...
                          ^

In [1]:
import json
import os
import psycopg
from psycopg import sql
import csv

def load_json(filepath):
    """Load JSON data from the specified filepath."""
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)

def write_to_csv(data, headers, filename):
    """Write data to CSV file located in the /tmp directory."""
    csv_path = os.path.join("/tmp", filename)
    with open(csv_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        total_rows = len(data)
        for index, row in enumerate(data):
            writer.writerow(row)
            if index % 100 == 0 or index == total_rows - 1:  # Update progress every 100 rows or at the end
                print(f"Writing {filename}: {index + 1}/{total_rows} rows completed.", end='\r')
        print()  # for newline after progress
    return csv_path

def process_json_data(json_file):
    """Process a single JSON file to CSV."""
    print(f"Starting processing of {json_file}")
    data = load_json(os.path.join("../data", json_file))  # Adjust the path according to your data directory
    entity = json_file.replace('.json', '')
    headers = {
        'cards': ["code", "money", "create_time"],
        'passengers': ["name", "id_number", "phone_number", "gender", "district"],
        'rides': ["user", "start_station", "end_station", "price", "start_time", "end_time"],
        'lines': ["name", "start_time", "end_time", "mileage", "color", "first_opening", "url", "intro"],
        'line_details': ["line_name", "station_name"],
        'stations': ["name", "district", "intro", "chinese_name"],
        'bus_stations': ["name", "district"],
        'bus_line_details': ["bus_line_name", "bus_station_name"],
        'exits': ["station_name", "name", "textt"]
    }[entity]

    # Check if data is a dictionary of dictionaries and convert it to a list of dictionaries
    if isinstance(data, dict) and all(isinstance(value, dict) for value in data.values()):
        data = [{**value, 'name': key} for key, value in data.items()]

    formatted_data = [[item[field] for field in headers] for item in data]
    csv_path = write_to_csv(formatted_data, headers, f"{entity}.csv")
    print(f"Finished writing {entity}.csv")
    return entity, csv_path

def main():
    """Main function to setup database and process all JSON files to PostgreSQL using COPY."""
    conn = psycopg.connect(dbname="project1", user="superlucky", password="", host="localhost", port="5432")
    cur = conn.cursor()

    # Drop existing tables if they exist to prevent conflicts
    print("Cleaning up existing tables...")
    cur.execute("DROP TABLE IF EXISTS rides, cards, passengers, line_details, lines, exits, bus_stations, stations, bus_line_details, bus_lines CASCADE;")

    # SQL command setup to create necessary tables
    print("Creating new tables...")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cards(code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'), money MONEY NOT NULL, create_time TIMESTAMP NOT NULL);
        CREATE TABLE IF NOT EXISTS passengers (name VARCHAR(255) NOT NULL, id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'), phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'), gender VARCHAR(255) NOT NULL, district VARCHAR(255) NOT NULL);
        CREATE TABLE IF NOT EXISTS rides (rail_user VARCHAR(255), start_station VARCHAR(255), end_station VARCHAR(255), price DOUBLE PRECISION, start_time TIMESTAMP, end_time TIMESTAMP);
        CREATE TABLE IF NOT EXISTS lines (name VARCHAR(255) PRIMARY KEY, start_time TIME, end_time TIME, mileage DOUBLE PRECISION, color VARCHAR(255), first_opening DATE, url TEXT, intro TEXT);
        CREATE TABLE IF NOT EXISTS line_details (line_name VARCHAR(255), station_name VARCHAR(255));
        CREATE TABLE IF NOT EXISTS stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255), intro TEXT, chinese_name VARCHAR(255));
        CREATE TABLE IF NOT EXISTS bus_stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255));
        CREATE TABLE IF NOT EXISTS bus_lines (name VARCHAR(255) PRIMARY KEY);
        CREATE TABLE IF NOT EXISTS bus_line_details (bus_line_name VARCHAR(255), bus_station_name VARCHAR(255), FOREIGN KEY (bus_line_name) REFERENCES bus_lines(name), FOREIGN KEY (bus_station_name) REFERENCES bus_stations(name));
        CREATE TABLE IF NOT EXISTS exits (station_name VARCHAR(255), name VARCHAR(255), textt TEXT, FOREIGN KEY (station_name) REFERENCES stations(name), PRIMARY KEY (station_name, name));
    """)
    print("Tables created successfully.")

    # Process each JSON file and copy the data to the database
    json_files = ["cards.json", "passengers.json", "rides.json", "lines.json", "stations.json"]
    for json_file in json_files:
        entity, csv_path = process_json_data(json_file)
        print(f"Copying {entity} data to PostgreSQL...")
        cur.execute(sql.SQL(f"COPY {entity} FROM '{csv_path}' WITH (FORMAT CSV, HEADER, DELIMITER ',')"))
        print(f"{entity} data copied successfully.")
    # Specific operations for bus_stations to handle potential conflicts
    print("Handling special cases for bus_stations...")
    cur.execute("""
        CREATE TEMP TABLE tmp_bus_stations AS SELECT * FROM bus_stations LIMIT 0;
        COPY tmp_bus_stations FROM '/tmp/bus_stations.csv' WITH (FORMAT CSV, HEADER, DELIMITER ',');
        INSERT INTO bus_stations SELECT * FROM tmp_bus_stations ON CONFLICT DO NOTHING;
        INSERT INTO bus_lines SELECT DISTINCT bus_line_name FROM bus_line_details;
    """)
    print("Special cases handled.")

    conn.commit()
    cur.close()
    conn.close()
    print("All data processed and uploaded successfully.")

if __name__ == "__main__":
    main()


Cleaning up existing tables...
Creating new tables...
Tables created successfully.
Starting processing of cards.json
Writing cards.csv: 10000/10000 rows completed.
Finished writing cards.csv
Copying cards data to PostgreSQL...
cards data copied successfully.
Starting processing of passengers.json
Writing passengers.csv: 10000/10000 rows completed.
Finished writing passengers.csv
Copying passengers data to PostgreSQL...
passengers data copied successfully.
Starting processing of rides.json
Writing rides.csv: 100000/100000 rows completed.
Finished writing rides.csv
Copying rides data to PostgreSQL...
rides data copied successfully.
Starting processing of lines.json
Writing lines.csv: 16/16 rows completed.
Finished writing lines.csv
Copying lines data to PostgreSQL...
lines data copied successfully.
Starting processing of stations.json
Writing stations.csv: 306/306 rows completed.
Finished writing stations.csv
Copying stations data to PostgreSQL...
stations data copied successfully.
Handl

UndefinedFile: could not open file "/tmp/bus_stations.csv" for reading: No such file or directory
HINT:  COPY FROM instructs the PostgreSQL server process to read a file. You may want a client-side facility such as psql's \copy.

In [3]:
import json
import os
import psycopg
from psycopg import sql
import csv

def load_json(filepath):
    """Load JSON data from the specified filepath."""
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)

def write_to_csv(data, headers, filename):
    """Write data to CSV file located in the /tmp directory."""
    csv_path = os.path.join("/tmp", filename)
    with open(csv_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        total_rows = len(data)
        for index, row in enumerate(data):
            writer.writerow(row)
            if index % 100 == 0 or index == total_rows - 1:  # Update progress every 100 rows or at the end
                print(f"Writing {filename}: {index + 1}/{total_rows} rows completed.", end='\r')
        print()  # for newline after progress
    return csv_path

def process_json_data(json_file):
    """Process a single JSON file to CSV."""
    print(f"Starting processing of {json_file}")
    data = load_json(os.path.join("../data", json_file))  # Adjust the path according to your data directory
    entity = json_file.replace('.json', '')
    headers = {
        'cards': ["code", "money", "create_time"],
        'passengers': ["name", "id_number", "phone_number", "gender", "district"],
        'rides': ["rail_user", "start_station", "end_station", "price", "start_time", "end_time"],
        'lines': ["name", "start_time", "end_time", "mileage", "color", "first_opening", "url", "intro"],
        'stations': ["name", "district", "intro", "chinese_name"],
        'exits': ["station_name", "name", "textt"]
    }[entity]

    formatted_data = [[item[field] for field in headers] for item in data]
    csv_path = write_to_csv(formatted_data, headers, f"{entity}.csv")
    print(f"Finished writing {entity}.csv")
       # Check if data is a dictionary of dictionaries and convert it to a list of dictionaries
    if isinstance(data, dict) and all(isinstance(value, dict) for value in data.values()):
        data = [{**value, 'name': key} for key, value in data.items()]

    formatted_data = [[item[field] for field in headers] for item in data]
    csv_path = write_to_csv(formatted_data, headers, f"{entity}.csv")
    print(f"Finished writing {entity}.csv")

    return entity, csv_path

def main():
    """Main function to setup database and process all JSON files to PostgreSQL using COPY."""
    conn = psycopg.connect(dbname="project1", user="superlucky", password="password", host="localhost", port="5432")
    cur = conn.cursor()

    # Drop existing tables if they exist to prevent conflicts
    print("Cleaning up existing tables...")
    cur.execute("DROP TABLE IF EXISTS rides, cards, passengers, line_details, lines, exits, bus_stations, stations, bus_line_details, bus_lines CASCADE;")

    # SQL command setup to create necessary tables
    print("Creating new tables...")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS cards(code VARCHAR(9) PRIMARY KEY CHECK (code ~ '^[0-9]{9}$'), money MONEY NOT NULL, create_time TIMESTAMP NOT NULL);
        CREATE TABLE IF NOT EXISTS passengers (name VARCHAR(255) NOT NULL, id_number VARCHAR(18) PRIMARY KEY NOT NULL CHECK (id_number ~ '^[0-9]{17}[0-9X]?$'), phone_number VARCHAR(11) NOT NULL CHECK (phone_number ~ '^[0-9]{11}$'), gender VARCHAR(255) NOT NULL, district VARCHAR(255) NOT NULL);
        CREATE TABLE IF NOT EXISTS rides (rail_user VARCHAR(255), start_station VARCHAR(255), end_station VARCHAR(255), price DOUBLE PRECISION, start_time TIMESTAMP, end_time TIMESTAMP);
        CREATE TABLE IF NOT EXISTS lines (name VARCHAR(255) PRIMARY KEY, start_time TIME, end_time TIME, mileage DOUBLE PRECISION, color VARCHAR(255), first_opening DATE, url TEXT, intro TEXT);
        CREATE TABLE IF NOT EXISTS stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255), intro TEXT, chinese_name VARCHAR(255));
        CREATE TABLE IF NOT EXISTS exits (station_name VARCHAR(255), name VARCHAR(255), textt TEXT, FOREIGN KEY (station_name) REFERENCES stations(name), PRIMARY KEY (station_name, name));
        CREATE TABLE IF NOT EXISTS bus_stations (name VARCHAR(255) PRIMARY KEY, district VARCHAR(255));
        CREATE TABLE IF NOT EXISTS bus_lines (name VARCHAR(255) PRIMARY KEY);
        CREATE TABLE IF NOT EXISTS bus_line_details (bus_line_name VARCHAR(255), bus_station_name VARCHAR(255), FOREIGN KEY (bus_line_name) REFERENCES bus_lines(name), FOREIGN KEY (bus_station_name) REFERENCES bus_stations(name));
        CREATE TABLE IF NOT EXISTS line_details (line_name VARCHAR(255), station_name VARCHAR(255), FOREIGN KEY (line_name) REFERENCES lines(name), FOREIGN KEY (station_name) REFERENCES stations(name));
    """)
    print("Tables created successfully.")

    # Process each JSON file and copy the data to the database
    json_files = ["cards.json", "passengers.json", "rides.json", "lines.json", "stations.json", "exits.json"]
    for json_file in json_files:
        entity, csv_path = process_json_data(json_file)
        print(f"Copying {entity} data to PostgreSQL...")
        cur.execute(sql.SQL(f"COPY {sql.Identifier(entity)} FROM %s WITH (FORMAT CSV, HEADER, DELIMITER ',')"), (csv_path,))
        print(f"{entity} data copied successfully.")

    # Handling derived tables (line_details, bus_stations, bus_line_details) after other data is loaded
    print("Deriving data for line_details, bus_stations, and bus_line_details...")
    cur.execute("""
        INSERT INTO line_details (line_name, station_name)
        SELECT name, unnest(string_to_array(intro, ', ')) FROM lines;

        INSERT INTO bus_stations (name, district)
        SELECT DISTINCT unnest(string_to_array(intro, ', ')), district FROM stations;

        INSERT INTO bus_line_details (bus_line_name, bus_station_name)
        SELECT DISTINCT line_name, name FROM line_details, bus_stations WHERE station_name = name;
    """)
    print("Derived tables populated.")

    conn.commit()
    cur.close()
    conn.close()
    print("All data processed and uploaded successfully.")

if __name__ == "__main__":
    main()


Cleaning up existing tables...
Creating new tables...
Tables created successfully.
Starting processing of cards.json
Writing cards.csv: 10000/10000 rows completed.
Finished writing cards.csv
Writing cards.csv: 10000/10000 rows completed.
Finished writing cards.csv
Copying cards data to PostgreSQL...


SyntaxError: syntax error at or near "'cards'"
LINE 1: COPY Identifier('cards') FROM $1 WITH (FORMAT CSV, HEADER, D...
                        ^